### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
school_data_complete


,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12,Thomas High School,51,48,14,Independent,1635,1043130
39166,39166,Dawn Bell,F,10,Thomas High School,81,89,14,Independent,1635,1043130
39167,39167,Rebecca Tanner,F,9,Thomas High School,99,99,14,Independent,1635,1043130
39168,39168,Desiree Kidd,F,10,Thomas High School,72,77,14,Independent,1635,1043130


In [3]:
#school_data_grouped = school_data_complete.drop_duplicates(subset = "School ID")

school_data_stats = school_data_complete.describe()
school_data_stats

,Student ID,year,reading_score,maths_score,School ID,size,budget
count,39170.000000,39170.000000,39170.000000,39170.000000,39170.000000,39170.000000,3.917000e+04
mean,19584.500000,10.359586,69.980138,70.338192,6.978172,3332.957110,2.117241e+06
std,11307.549359,1.102779,17.242346,16.910154,4.444329,1323.914069,8.749987e+05
min,0.000000,9.000000,39.000000,39.000000,0.000000,427.000000,2.480870e+05
25%,9792.250000,9.000000,55.000000,56.000000,3.000000,1858.000000,1.081356e+06
50%,19584.500000,10.000000,70.000000,70.000000,7.000000,2949.000000,1.910635e+06
75%,29376.750000,11.000000,85.000000,85.000000,11.000000,4635.000000,3.022020e+06
max,39169.000000,12.000000,99.000000,99.000000,14.000000,4976.000000,3.124928e+06


In [4]:
def get_passing_stats(df):
  num_passing_math = df[df['maths_score'] >= 50].shape[0]
  pct_passing_math = num_passing_math / df.shape[0]

  num_passing_reading = df[df['reading_score'] >= 50].shape[0]
  pct_passing_reading = num_passing_reading / df.shape[0]

  num_passing_overall = df[(df['maths_score'] >= 50) & (df['reading_score'] >= 50)].shape[0]
  pct_passing_overall = num_passing_overall / df.shape[0]

  return [
    pct_passing_math,
    pct_passing_reading,
    pct_passing_overall
  ]



In [13]:
nStudents = school_data_complete["Student ID"].nunique()
nSchools = school_data_complete["School ID"].nunique()
totalBudget = school_data["budget"].sum()
avgReading = school_data_stats.iat[1, 2]
avgMaths = school_data_stats.iat[1, 3]

stats = get_passing_stats(school_data_complete)
pctMathPass, pctReadingPass, pctBothPass = stats[0], stats[1], stats[2]


avgMaths


70.33819249425581

In [24]:
#TODO Format
lga_summary = {
  'Total Schools': nSchools,
  'Total Students': nStudents,
  'Total Budget': totalBudget,
  'Average Maths Score': avgMaths,
  'Average Reading Score': avgReading,
  '% Passing Maths': pctMathPass,
  '% Passing Reading': pctReadingPass,
  '% Overall Passing': pctBothPass
}

lga_summary_df = pd.DataFrame(lga_summary, index=[0])
lga_summary_df

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,39170,24649428,70.338192,69.980138,0.860786,0.844269,0.728083


In [25]:
grouped_school_complete = school_data_complete.groupby(by = 'School ID')

In [68]:
school_summary_data = []
columns = [
  'School Type', 
  'Total Students', 
  'Total School Budget', 
  'Per Student Budget', 
  'Average Maths Score', 
  'Average Reading Score', 
  '% Passing Maths', 
  '% Passing Reading', 
  '% Overall Passing'
]

index = [
  'Huang High School',
  'Figueroa High School',
  'Shelton High School',
  'Hernandez High School',
  'Griffin High School',
  'Wilson High School',
  'Cabrera High School',
  'Bailey High School',
  'Holden High School',
  'Pena High School',
  'Wright High School',
  'Rodriguez High School',
  'Johnson High School',
  'Ford High School',
  'Thomas High School'
]

for name, group in grouped_school_complete:
  school_type = group.iloc[0]['type']
  total_students = group.shape[0]
  total_budget = group.iloc[0]['budget']
  per_student_budget = total_budget / total_students
  avg_math_score = group['maths_score'].mean()
  avg_reading_score = group['reading_score'].mean()
  passing_stats = get_passing_stats(group)
  
  school_summary_data.append([
    school_type,
    total_students,
    total_budget,
    per_student_budget,
    avg_math_score,
    avg_reading_score,
    passing_stats[0],
    passing_stats[1],
    passing_stats[2]
  ])

school_summary_df = pd.DataFrame(school_summary_data, index = index, columns = columns)


In [212]:
school_summary_df['% Passing Maths'] = round(school_summary_df['% Passing Maths']*100, 2)
school_summary_df['% Passing Reading'] = round(school_summary_df['% Passing Reading']*100, 2)
school_summary_df['% Overall Passing'] = round(school_summary_df['% Overall Passing']*100, 2)
school_summary_df[['Average Maths Score', 'Average Reading Score']] = round(school_summary_df[['Average Maths Score', 'Average Reading Score']], 2)




In [213]:
school_summary_df.sort_index()
#TODO Format % and budget to $

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing,Spending Range,Spending Range (per student)
Bailey High School,Government,4976,3124928,628.0,72.35,71.01,91.64,87.38,80.08,$585-630,$585-630
Cabrera High School,Independent,1858,1081356,582.0,71.66,71.36,90.85,89.07,80.79,<$585,<$585
Figueroa High School,Government,2949,1884411,639.0,68.70,69.08,81.65,82.81,67.65,$630-645,$630-645
Ford High School,Government,2739,1763916,644.0,69.09,69.57,82.44,82.22,67.47,$630-645,$630-645
Griffin High School,Independent,1468,917500,625.0,71.79,71.25,91.21,88.49,81.34,$585-630,$585-630
Hernandez High School,Government,4635,3022020,652.0,68.87,69.19,80.95,81.88,66.36,$645-680,$645-680
Holden High School,Independent,427,248087,581.0,72.58,71.66,89.93,88.52,78.92,<$585,<$585
Huang High School,Government,2917,1910635,655.0,68.94,68.91,81.69,81.45,66.71,$645-680,$645-680
Johnson High School,Government,4761,3094650,650.0,68.84,69.04,82.06,81.98,67.19,$645-680,$645-680
Pena High School,Independent,962,585858,609.0,72.09,71.61,91.68,86.59,79.21,$585-630,$585-630


In [70]:
school_summary_df.sort_values('% Overall Passing', ascending = False).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Griffin High School,Independent,1468,917500,625.0,71.788147,71.245232,0.912125,0.884877,0.813351
Cabrera High School,Independent,1858,1081356,582.0,71.657158,71.359526,0.908504,0.890743,0.807858
Bailey High School,Government,4976,3124928,628.0,72.352894,71.008842,0.916399,0.873794,0.800844
Wright High School,Independent,1800,1049400,583.0,72.047222,70.969444,0.917778,0.866667,0.797222
Rodriguez High School,Government,3999,2547363,637.0,72.047762,70.935984,0.907977,0.873968,0.794199


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [67]:
school_summary_df.sort_values('% Overall Passing', ascending = True).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Hernandez High School,Government,4635,3022020,652.0,68.874865,69.186408,0.809493,0.818770,0.663646
Huang High School,Government,2917,1910635,655.0,68.935207,68.910525,0.816935,0.814535,0.667124
Johnson High School,Government,4761,3094650,650.0,68.843100,69.039277,0.820626,0.819786,0.671918
Wilson High School,Independent,2283,1319574,578.0,69.170828,68.876916,0.827858,0.812965,0.674551
Ford High School,Government,2739,1763916,644.0,69.091274,69.572472,0.824388,0.822198,0.674699


## Maths Scores by Year

In [183]:
grouped_school_year = school_data_complete.groupby(['year','school_name'])

maths_averages = {}

for name, group in grouped_school_year:
    school_name, year = name
    avg_score = group['maths_score'].mean()
    if school_name not in maths_averages:
        maths_averages[school_name] = {year: avg_score}
    else:
        maths_averages[school_name][year] = avg_score

maths_averages_df = pd.DataFrame(maths_averages)
maths_averages_df.rename(columns={'index': 'school_name'}, inplace=True)
maths_averages_df.rename(columns=lambda x: f"Year {x}", inplace=True)
maths_averages_df


,Year 9,Year 10,Year 11,Year 12
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393
Pena High School,71.996364,72.396000,72.523438,71.187845


In [184]:
reading_averages = {}

for name, group in grouped_school_year:
    school_name, year = name
    avg_score = group['reading_score'].mean()
    if school_name not in reading_averages:
        reading_averages[school_name] = {year: avg_score}
    else:
        reading_averages[school_name][year] = avg_score

reading_averages_df = pd.DataFrame(reading_averages)
reading_averages_df.rename(columns={'index': 'school_name'}, inplace=True)
reading_averages_df.rename(columns=lambda x: f"Year {x}", inplace=True)
reading_averages_df

,Year 9,Year 10,Year 11,Year 12
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521
Pena High School,70.949091,72.324000,71.703125,71.513812


## Scores by School Spending

In [186]:
#school_summary_df

spending_bins = [0, 585, 630, 645, 680]
spending_names = ["<$585", "$585-630", "$630-645", "$645-680"]


In [216]:
school_summary_df['Spending Range (per student)'] = pd.cut(school_summary_df['Per Student Budget'], bins = spending_bins, labels = spending_names, include_lowest = True)
school_summary_df


,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing,Spending Range (per student)
Huang High School,Government,2917,1910635,655.0,68.94,68.91,81.69,81.45,66.71,$645-680
Figueroa High School,Government,2949,1884411,639.0,68.70,69.08,81.65,82.81,67.65,$630-645
Shelton High School,Independent,1761,1056600,600.0,72.03,70.26,91.54,86.71,78.88,$585-630
Hernandez High School,Government,4635,3022020,652.0,68.87,69.19,80.95,81.88,66.36,$645-680
Griffin High School,Independent,1468,917500,625.0,71.79,71.25,91.21,88.49,81.34,$585-630
Wilson High School,Independent,2283,1319574,578.0,69.17,68.88,82.79,81.30,67.46,<$585
Cabrera High School,Independent,1858,1081356,582.0,71.66,71.36,90.85,89.07,80.79,<$585
Bailey High School,Government,4976,3124928,628.0,72.35,71.01,91.64,87.38,80.08,$585-630
Holden High School,Independent,427,248087,581.0,72.58,71.66,89.93,88.52,78.92,<$585
Pena High School,Independent,962,585858,609.0,72.09,71.61,91.68,86.59,79.21,$585-630


In [217]:
grouped_summary_ranges = school_summary_df.groupby('Spending Range (per student)')
mean_summary_ranges = grouped_summary_ranges.mean()
mean_summary_ranges

,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Range (per student),,,,,,,,
<$585,1592.000000,9.246042e+05,581.000000,71.365000,70.717500,88.837500,86.3900,76.722500
$585-630,2291.750000,1.421222e+06,615.500000,72.065000,71.032500,91.517500,87.2925,79.877500
$630-645,2830.500000,1.809705e+06,639.500000,69.855000,69.840000,84.685000,83.7650,71.005000
$645-680,4104.333333,2.675768e+06,652.333333,68.883333,69.046667,81.566667,81.7700,66.753333


In [230]:
spending_summary = mean_summary_ranges.iloc[:, [3 , 4, 5, 6, 7]]
spending_summary.round(2)


,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Range (per student),,,,,
<$585,71.36,70.72,88.84,86.39,76.72
$585-630,72.06,71.03,91.52,87.29,79.88
$630-645,69.86,69.84,84.68,83.76,71.00
$645-680,68.88,69.05,81.57,81.77,66.75


## Scores by School Size

In [224]:
size_bins = [0, 1000, 2000, 5000]
size_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [225]:
school_summary_df['School Size'] = pd.cut(school_summary_df['Total Students'], bins = size_bins, labels = size_names, include_lowest = True)
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing,Spending Range (per student),School Size
Huang High School,Government,2917,1910635,655.0,68.94,68.91,81.69,81.45,66.71,$645-680,Large (2000-5000)
Figueroa High School,Government,2949,1884411,639.0,68.70,69.08,81.65,82.81,67.65,$630-645,Large (2000-5000)
Shelton High School,Independent,1761,1056600,600.0,72.03,70.26,91.54,86.71,78.88,$585-630,Medium (1000-2000)
Hernandez High School,Government,4635,3022020,652.0,68.87,69.19,80.95,81.88,66.36,$645-680,Large (2000-5000)
Griffin High School,Independent,1468,917500,625.0,71.79,71.25,91.21,88.49,81.34,$585-630,Medium (1000-2000)
Wilson High School,Independent,2283,1319574,578.0,69.17,68.88,82.79,81.30,67.46,<$585,Large (2000-5000)
Cabrera High School,Independent,1858,1081356,582.0,71.66,71.36,90.85,89.07,80.79,<$585,Medium (1000-2000)
Bailey High School,Government,4976,3124928,628.0,72.35,71.01,91.64,87.38,80.08,$585-630,Large (2000-5000)
Holden High School,Independent,427,248087,581.0,72.58,71.66,89.93,88.52,78.92,<$585,Small (<1000)
Pena High School,Independent,962,585858,609.0,72.09,71.61,91.68,86.59,79.21,$585-630,Small (<1000)


In [226]:
grouped_summary_size = school_summary_df.groupby('School Size')
mean_summary_size = grouped_summary_size.mean()
mean_summary_size

,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,,,,
Small (<1000),694.500,416972.500,595.000,72.33500,71.6350,90.8050,87.5550,79.0650
Medium (1000-2000),1704.400,1029597.200,605.600,71.42200,70.7220,89.8460,86.7140,78.0420
Large (2000-5000),3657.375,2333437.125,635.375,69.75125,69.5775,84.2525,83.3025,70.2925


In [229]:
mean_summary_size.iloc[:, [3 , 4, 5, 6, 7]].round(2)

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.34,71.63,90.80,87.56,79.06
Medium (1000-2000),71.42,70.72,89.85,86.71,78.04
Large (2000-5000),69.75,69.58,84.25,83.30,70.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [232]:
grouped_summary_type = school_summary_df.groupby('School Type')
mean_summary_type = grouped_summary_type.mean()
mean_summary_type.iloc[:, [3 , 4, 5, 6, 7]].round(2)

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.83,69.68,84.46,83.59,70.70
Independent,71.37,70.72,89.20,86.25,76.98
